<a href="https://colab.research.google.com/github/garden-government/RGR_TIMP_3/blob/main/RGR_TIMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Reshape

import keras

from pandas.core.sorting import lexsort_indexer

from PIL import Image
from numpy import asarray
import numpy

In [ ]:
'''
Создание тестовой и обучающей выборки для обучения нейронной сети

'''

how_many_letter = 5;
all_letter = ['а','б','в','г','д','е','ё','ж','з','и','й','к','л','м','н']

dir_learning = 'drive/MyDrive/learning'
how_many_lerning_image = 44

x_train_workpiece = []
for j in range (how_many_letter):
  print(all_letter[j])
  for i in range (how_many_lerning_image):
    x_train_workpiece.append(asarray( Image.open(dir_learning + '/' + all_letter[j] + '/'+ all_letter[j] + str(i+1) + '.png','r', formats=None)))
    
x_train = numpy.array (x_train_workpiece)
x_train = x_train / 255


y_train_workpiece = []
for number_of_letter in range (how_many_letter):
  for i in range (how_many_lerning_image):
    y = [number_of_letter]
    y_train_workpiece.append(asarray(y)) 

y_train = numpy.array (y_train_workpiece)
y_train_cat = keras.utils.to_categorical(y_train,how_many_letter)


In [ ]:
dir_check = 'drive/MyDrive/check'
how_many_test_image = 4

x_test_workpiece = []
for j in range (how_many_letter):
  print(all_letter[j])
  for i in range (how_many_test_image):
    x_test_workpiece.append(asarray( Image.open(dir_check + '/' + all_letter[j] + '/'+ all_letter[j] + str(i+45) + '.png','r', formats=None)))

x_test = numpy.array (x_test_workpiece)
x_test = x_test / 255


y_test_workpiece = []
for number_of_letter in range (how_many_letter):
  for i in range (how_many_test_image):
    y = [number_of_letter]
    y_test_workpiece.append(asarray(y)) 

y_test = numpy.array (y_test_workpiece)
y_test_cat = keras.utils.to_categorical(y_test,how_many_letter)

In [ ]:
'''
Нейронная сеть с использованием библиотеки keras, предназначенной специально для создания нейронных сетей и их обучения

'''

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(64, 64, 4)))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(how_many_letter, activation="softmax"))


In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])


In [ ]:
his = model.fit(x_train, y_train_cat, batch_size=1, epochs=1, validation_split=0.2)


In [ ]:
model.evaluate(x_test,y_test_cat)


In [ ]:
'''
Нейронная сеть с использованием библиотеки numpy для расчета операций с матрицами и некоторыми вспмоогательными функиями

'''

def relu(t):
    return numpy.maximum(t, 0)

def relu_derivative(t):
    return (t >= 0).astype(float)

def softmax(t):
  out = numpy.exp(t)
  return out / numpy.sum(out)

def sparse_cross_entropy(z, y):
  return -numpy.log(z[0, y])

ALPHA = 0.0002
EPOHS = 5

WIDTH_PAINT = 64
HEIGHT_PAINT = 64

LR_WIDTH_1 = 4096
LR_WIDTH_2 = 2222
LR_OUT = how_many_letter

W1 = numpy.random.rand(LR_WIDTH_1, LR_WIDTH_2)
b1 = numpy.random.rand(1, LR_WIDTH_2)
W2 = numpy.random.rand(LR_WIDTH_2, LR_OUT)
b2 = numpy.random.rand(1, LR_OUT)

W1 = (W1 - 0.5) * 2 * numpy.sqrt(1/LR_WIDTH_1)
b1 = (b1 - 0.5) * 2 * numpy.sqrt(1/LR_WIDTH_1)
W2 = (W2 - 0.5) * 2 * numpy.sqrt(1/LR_WIDTH_2)
b2 = (b2 - 0.5) * 2 * numpy.sqrt(1/LR_WIDTH_2)

popatka = 0
lose = 0

for number_era in range(EPOHS):
  print("             Начало эпохи под номер:", number_era)
  for number_image in range(len(x_train)):
    try:
      print(" Обработка изобрадения под номером: ", number_image)

      paint_full = x_train[number_image]
      paint_one_stream = paint_full[0:WIDTH_PAINT, 0:WIDTH_PAINT, 0:1]
      flatten_paint = paint_one_stream.flatten()

      FL_LR = numpy.asarray([flatten_paint])

      t1 = FL_LR @ W1 + b1
      h1 = relu(t1) 

      t2 = h1 @ W2 + b2
      z = softmax(t2)

      #в обратную сторону
      y_full = y_train_cat[number_image]
      dE_dt2 = z - y_full

      dE_dW2 = h1.T @ dE_dt2                                     
      dE_db2 = dE_dt2                                           

      dE_dh1 = dE_dt2 @ W2.T
      dE_dt1 = dE_dh1 * relu_derivative(t1)

      dE_dW1 = FL_LR.T @ dE_dt1                                  
      dE_db1 = numpy.sum(dE_dt1, axis=0, keepdims=True)          


      W1 -= ALPHA * dE_dW1
      b1 -= ALPHA * dE_db1
      W2 -= ALPHA * dE_dW2
      b2 -= ALPHA * dE_db2

      popatka += 1
      if numpy.argmax(z[0]) != y_train[number_image,0]:
        lose += 1
    except:
      print(" При обработке изобрадения под номером ", number_image, " возникла ошибка!")
      print(popatka, lose)


In [ ]:
#   Тестирование
popatka = 0
lose = 0

for number_image in range(len(x_test)):
  try:
      print("Тест изобрадения под номером: ", number_image)

      paint_full = x_test[number_image]
      paint_one_stream = paint_full[0:WIDTH_PAINT, 0:WIDTH_PAINT, 0:1]
      flatten_paint = paint_one_stream.flatten()

      FL_LR = numpy.asarray([flatten_paint])
      t1 = FL_LR @ W1 + b1
      h1 = relu(t1) 
      t2 = h1 @ W2 + b2
      z = softmax(t2)         

      popatka += 1
      if numpy.argmax(z[0]) != y_test[number_image,0]:
        lose += 1
  except:
      print(" При тесте изобрадения под номером ", number_image, " возникла ошибка!")
    
print(" Тест завершён!")
print("Количество тестовых ихображений: ",popatka)
print("Количество правильных ответов: ", lose)

In [ ]:
# save
'''
from pandas.core.sorting import lexsort_indexer
import numpy

def relu(t):
    return numpy.maximum(t, 0)

def relu_deriv(t):
    return (t >= 0).astype(float)

def to_full(y, num_classes):
    y_full = numpy.zeros((1, num_classes))
    y_full[0, y] = 1
    return y_full

def inver_matrix (matrix):
  result_matrix = numpy.random.rand(len(matrix),len(matrix[0]))
  for i in range(len(matrix)):
    for j in range(len(matrix[0])):
      result_matrix[i, j] = matrix[len(matrix) - i -1, len(matrix[0]) - j - 1]
  return result_matrix

def softmax(t):
  out = numpy.exp(t)
  return out / numpy.sum(out)

def sparse_cross_entropy(z, y):
  return -numpy.log(z[0, y])

ALPHA = 0.0002

popatka = 0
lose = 0

WIDTH_PAINT = 64
HEIGHT_PAINT = 64

WIDTH_MASKA_1_2 = 5
HEIGHT_MASKA_1_2 = 5
number_of_mask_1 = 5
number_of_mask_2 = 3

maska = numpy.random.rand((number_of_mask_1), (WIDTH_MASKA_1_2), (WIDTH_MASKA_1_2))
maska_b1 = numpy.random.rand(1, number_of_mask_1)
maska2 = numpy.random.rand((number_of_mask_2), (WIDTH_MASKA_1_2), (WIDTH_MASKA_1_2))
maska_b2 = numpy.random.rand(1, number_of_mask_2)

maska = (maska - 0.5) * 2 * numpy.sqrt(1/(WIDTH_MASKA_1_2 * WIDTH_MASKA_1_2))
maska_b1 = (maska_b1 - 0.5) * 2 * numpy.sqrt(1/number_of_mask_1)
maska2 = (maska2 - 0.5) * 2 * numpy.sqrt(1/number_of_mask_2)
maska_b2 = (maska_b2 - 0.5) * 2 * numpy.sqrt(1/(WIDTH_MASKA_1_2 * WIDTH_MASKA_1_2))

LR_WIDTH_1 = 43740
LR_WIDTH_2 = 100
LR_OUT = 5

W1 = numpy.random.rand(LR_WIDTH_1, LR_WIDTH_2)
b1 = numpy.random.rand(1, LR_WIDTH_2)
W2 = numpy.random.rand(LR_WIDTH_2, LR_OUT)
b2 = numpy.random.rand(1, LR_OUT)

W1 = (W1 - 0.5) * 2 * numpy.sqrt(1/LR_WIDTH_1)
b1 = (b1 - 0.5) * 2 * numpy.sqrt(1/LR_WIDTH_1)
W2 = (W2 - 0.5) * 2 * numpy.sqrt(1/LR_WIDTH_2)
b2 = (b2 - 0.5) * 2 * numpy.sqrt(1/LR_WIDTH_2)

print(W1,b1,W2,b2)
print('////////////')
print(maska,maska2)

for number_image in range(len(x_train)): #len(x_train)
  #try:
    print(number_image, " обработка изображения")
    paint_full = x_train[number_image]
    paint = paint_full[0:63, 0:63, 0:1]
    
    after_mask_1 = numpy.arange(59*59*5).reshape((5), (59), (59))

    for number_mask in range(number_of_mask_1):
      #переоб отдельных кусков
      for j in range(len(paint) - HEIGHT_MASKA_1_2 + 1):
        for i in range(len(paint[0]) - WIDTH_MASKA_1_2 + 1):

          #перемножение с маской и складывание
          after_mask_1[number_mask,j,i] = numpy.sum( numpy.multiply( paint[j:(j + HEIGHT_MASKA_1_2), i:(i + WIDTH_MASKA_1_2)], maska[number_mask] ) )

    for number_b in range(number_of_mask_1):
      after_mask_1[number_b] = after_mask_1[number_b] + maska_b1[0 ,number_b]


    after_mask_2 = numpy.arange(54*54*15).reshape((15), (54), (54))

    for number_lr in range(len(after_mask_1)):
      lr = after_mask_1[number_lr]

      for number_mask in range(number_of_mask_2):
        #переоб отдельных кусков
        for j in range(len(lr) - HEIGHT_MASKA_1_2):
          for i in range(len(lr[0]) - WIDTH_MASKA_1_2):

            #перемножение с маской и складывание
            after_mask_2[number_mask + number_lr * number_of_mask_2,j,i] = numpy.sum( numpy.multiply( lr[j:(j + HEIGHT_MASKA_1_2), i:(i + WIDTH_MASKA_1_2)], maska2[number_mask] ) )
    
    for number_b in range(number_of_mask_2):
      after_mask_2[number_b] = after_mask_2[number_b * 3 + 0] + maska_b1[0 ,number_b]
      after_mask_2[number_b] = after_mask_2[number_b * 3 + 1] + maska_b1[0 ,number_b]
      after_mask_2[number_b] = after_mask_2[number_b * 3 + 2] + maska_b1[0 ,number_b]

    flatten_lr = after_mask_2.flatten()
    FL_LR = numpy.asarray([flatten_lr])
    t1 = FL_LR @ W1 + b1
    h1 = relu(t1) 
    t2 = h1 @ W2 + b2
    z = softmax(t2)
    E = sparse_cross_entropy(z, y_train[number_image,0])

    popatka = popatka + 1
    if numpy.argmax(z[0]) != y_train[number_image,0]:
      lose = lose + 1


    #в обратную сторону

    y_full = to_full(y_train[number_image,0], LR_OUT)

    dE_dt2 = z - y_full
    dE_dW2 = h1.T @ dE_dt2                                     ##############
    dE_db2 =  dE_dt2                                           ##############

    dE_dh1 = dE_dt2 @ W2.T
    dE_dt1 = dE_dh1 * relu_deriv(t1)
    dE_dW1 = FL_LR.T @ dE_dt1                                  #############
    dE_db1 = numpy.sum(dE_dt1, axis=0, keepdims=True)          #############

    dE_dh0 = dE_dt1 @ W1.T


    maskaE = numpy.random.rand(15, 54, 54)

    for i in range(len(dE_dh0[0])):
      lr_index = i // 2916
      line_index =  (i % 2916) // 54
      colum_index = (i % 2916) % 54
      maskaE[lr_index, line_index, colum_index] = dE_dh0[0, i]

    b2_before_mask = numpy.random.rand(1, 3)

    for j in range (3):
      b2_before_mask[0, j] = numpy.sum(maskaE[j * 3 + 0])     ################
      b2_before_mask[0, j] += numpy.sum(maskaE[j * 3 + 1])
      b2_before_mask[0, j] += numpy.sum(maskaE[j * 3 + 2])

    maskaWE2 = numpy.random.rand(3, 5, 5)                      

    for number_mask in range(3):
      for index_in_mask in range(25):
        line_index = index_in_mask // 5
        coulm_index = index_in_mask % 5

        summE = 0
        for index_lr_mask in range(5):
          lr_help = after_mask_1 [index_lr_mask]   
          summE += numpy.sum (lr_help[coulm_index: coulm_index + 54, line_index: line_index + 54] * maskaE[index_lr_mask + number_mask * 5])
        maskaWE2[number_mask, line_index, coulm_index] = summE    ################


    lr_before_mask = numpy.random.rand(5, 59, 59)

    for number_lr in range(5):

      for i in range(len(lr_before_mask[0])):
        for j in range(len(lr_before_mask[0,0]) - 1):

          sum_help = 0
          for number_help in range(3):
            help_maska = numpy.pad(maskaE[number_lr * 3 + number_help], ((4,5),(4,5)), 'constant')
            sum_help += numpy.sum (help_maska[i:i+5,j:j+5] * inver_matrix(maskaWE2[number_help]))
          lr_before_mask[number_lr, i, j] = sum_help

    b1_before_mask = numpy.random.rand(1, 5)
    for j in range (5):
      b1_before_mask[0, j] = numpy.sum(lr_before_mask[j])       ####################
    
    maskaWE1 = numpy.random.rand(5, 5, 5)

    for number_mask in range(5):
      for index_in_mask in range(25):
        line_index = index_in_mask // 5
        coulm_index = index_in_mask % 5

        summE = 0
        for index_lr_mask in range(1):
          lr_help = paint_full[0:63, 0:63, 0:1]
          summE += numpy.sum (lr_help[coulm_index: coulm_index + 59, line_index: line_index + 59] * lr_before_mask[number_mask])
        maskaWE1[number_mask, line_index, coulm_index] = summE     ###############

    #спуск

    W1 = W1 - ALPHA * dE_dW1
    b1 = b1 - ALPHA * dE_db1
    W2 = W2 - ALPHA * dE_dW2
    b2 = b2 - ALPHA * dE_db2

    maska = maska - ALPHA * maskaWE1
    maska2 = maska2 - ALPHA * maskaWE2
    maska_b1 = maska_b1 - ALPHA * b1_before_mask
    maska_b2 = maska_b2 - ALPHA * b2_before_mask
  #except:
    print(number_image, "ERROR")
    print(W1,b1,W2,b2)
    print('////////////')
    print(maska,maska2)

print(popatka, lose)


#arr = numpy.array([[1,2,3,4],[4,5,6,7],[7,8,9,10],[17,18,19,110]])
#print(paint)
#a = numpy.array([[1,2],[3,4]])
#b = numpy.array([[5,6],[7,8]])
#numpy.multiply(a,b)
#print(numpy.multiply(a,b))
#print(numpy.sum(a))

#print(inver_matrix(numpy.array([[1,2,3],[4,5,6],[7,8,9]])))

#print(numpy.pad(numpy.array([[1,2,3],[4,5,6],[7,8,9]]), ((4,4),(4,4)), 'constant'))

#.flatten()
''' 
